##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/video/transfer_learning_with_movinet"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/video/transfer_learning_with_movinet.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

# MoViNet을 사용한 비디오 분류 전이 학습

MoViNets(Mobile Video Networks)는 스트리밍 비디오에 대한 추론을 지원하는 효율적인 비디오 분류 모델군을 제공합니다. 이 튜토리얼에서는 사전 훈련된 MoViNet 모델을 사용하여 [UCF101 데이터세트](https://www.crcv.ucf.edu/data/UCF101.php)로부터 특히 동작 인식 작업을 위해 비디오를 분류합니다. 사전 훈련된 모델은 이전에 더 큰 데이터세트에서 훈련된 저장된 네트워크입니다. Kondratyuk, D. 등(2021)의 [MoViNets: 효율적인 비디오 인식을 위한 모바일 비디오 네트워크](https://arxiv.org/abs/2103.11511) 논문에서 MoViNets에 대한 자세한 내용을 확인할 수 있습니다. 이 튜토리얼에서는 다음을 수행합니다.

- 사전 훈련된 MoViNet 모델을 다운로드하는 방법을 알아봅니다.
- MoViNet 모델의 컨볼루션 기반을 동결하여 새로운 분류기가 있는 사전 훈련된 모델로 새 모델을 생성합니다.
- 분류기 헤드를 새 데이터세트의 레이블 수로 교체합니다.
- [UCF101 데이터세트](https://www.crcv.ucf.edu/data/UCF101.php)에서 전이 학습을 수행합니다.

이 튜토리얼에서 다운로드한 모델은 [official/projects/movinet](https://github.com/tensorflow/models/tree/master/official/projects/movinet)에서 가져온 것입니다. 이 리포지토리에는 TF Hub가 TensorFlow 2 SavedModel 형식으로 사용하는 MoViNet 모델 컬렉션이 포함되어 있습니다.

이 전이 학습 튜토리얼은 TensorFlow 비디오 튜토리얼 시리즈의 세 번째 부분입니다. 다른 세 개의 튜토리얼은 다음과 같습니다.

- [비디오 데이터 로드](https://www.tensorflow.org/tutorials/load_data/video): 이 튜토리얼은 이 문서에서 사용된 많은 코드를 설명합니다. 특히 `FrameGenerator` 클래스를 통해 데이터를 전처리하고 로드하는 방법에 대해 자세히 설명합니다.
- [비디오 분류를 위한 3D CNN 모델 구축](https://www.tensorflow.org/tutorials/video/video_classification): 이 튜토리얼에서는 3D 데이터의 공간적 및 시간적 측면을 분해하는 (2+1)D CNN을 사용합니다. MRI 스캔과 같은 체적 데이터를 사용하는 경우 (2+1)D CNN 대신 3D CNN을 사용하는 것이 좋습니다.
- [스트리밍 동작 인식을 위한 MoViNet](https://www.tensorflow.org/hub/tutorials/movinet): TF Hub에서 사용할 수 있는 MoViNet 모델에 익숙해집니다.

## 설정

우선 ZIP 파일의 내용을 검사하기 위한 [remotezip](https://github.com/gtsystem/python-remotezip), 진행률 표시줄을 사용하기 위한 [tqdm](https://github.com/tqdm/tqdm), 비디오 파일을 처리하기 위한 [OpenCV](https://opencv.org/)(`opencv-python` 및 `opencv-python-headless`가 동일한 버전인지 확인) 및 사전 훈련된 MoViNet 모델을 다운로드하기 위한 TensorFlow 모델([`tf-models-official`](https://github.com/tensorflow/models/tree/master/official))을 포함하여 몇 가지 필요한 라이브러리를 설치하고 가져옵니다. TensorFlow 모델 패키지는 TensorFlow의 고급 API를 사용하는 모델 컬렉션입니다.

In [ ]:
!pip install remotezip tqdm opencv-python==4.5.2.52 opencv-python-headless==4.5.2.52 tf-models-official

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

## 데이터 로드하기

아래 숨겨진 셀은 UCF-101 데이터세트에서 데이터 조각을 다운로드하고 `tf.data.Dataset`에 로드하는 헬퍼 함수를 정의합니다. [비디오 데이터 로드 튜토리얼](https://www.tensorflow.org/tutorials/load_data/video)은 이 코드에 대한 자세한 연습을 제공합니다.

숨겨진 블록 끝에 있는 `FrameGenerator` 클래스는 여기에서 가장 중요한 유틸리티로, TensorFlow 데이터 파이프라인에 데이터를 공급할 수 있는 반복 가능한 객체를 생성합니다. 특히 이 클래스에는 인코딩된 레이블과 함께 비디오 프레임을 로드하는 Python 생성기가 포함되어 있습니다. 생성기(`__call__`) 함수는 `frames_from_video_file`에 의해 생성된 프레임 배열과 프레임 세트와 관련된 레이블의 원-핫 인코딩 벡터를 생성합니다.


In [ ]:
#@title 

def list_files_per_class(zip_url):
  """
    List the files in each class of the dataset given the zip URL.

    Args:
      zip_url: URL from which the files can be unzipped. 

    Return:
      files: List of files in each of the classes.
  """
  files = []
  with rz.RemoteZip(URL) as zip:
    for zip_info in zip.infolist():
      files.append(zip_info.filename)
  return files

def get_class(fname):
  """
    Retrieve the name of the class given a filename.

    Args:
      fname: Name of the file in the UCF101 dataset.

    Return:
      Class that the file belongs to.
  """
  return fname.split('_')[-3]

def get_files_per_class(files):
  """
    Retrieve the files that belong to each class. 

    Args:
      files: List of files in the dataset.

    Return:
      Dictionary of class names (key) and files (values).
  """
  files_for_class = collections.defaultdict(list)
  for fname in files:
    class_name = get_class(fname)
    files_for_class[class_name].append(fname)
  return files_for_class

def download_from_zip(zip_url, to_dir, file_names):
  """
    Download the contents of the zip file from the zip URL.

    Args:
      zip_url: Zip URL containing data.
      to_dir: Directory to download data to.
      file_names: Names of files to download.
  """
  with rz.RemoteZip(zip_url) as zip:
    for fn in tqdm.tqdm(file_names):
      class_name = get_class(fn)
      zip.extract(fn, str(to_dir / class_name))
      unzipped_file = to_dir / class_name / fn

      fn = pathlib.Path(fn).parts[-1]
      output_file = to_dir / class_name / fn
      unzipped_file.rename(output_file,)

def split_class_lists(files_for_class, count):
  """
    Returns the list of files belonging to a subset of data as well as the remainder of
    files that need to be downloaded.

    Args:
      files_for_class: Files belonging to a particular class of data.
      count: Number of files to download.

    Return:
      split_files: Files belonging to the subset of data.
      remainder: Dictionary of the remainder of files that need to be downloaded.
  """
  split_files = []
  remainder = {}
  for cls in files_for_class:
    split_files.extend(files_for_class[cls][:count])
    remainder[cls] = files_for_class[cls][count:]
  return split_files, remainder

def download_ufc_101_subset(zip_url, num_classes, splits, download_dir):
  """
    Download a subset of the UFC101 dataset and split them into various parts, such as
    training, validation, and test. 

    Args:
      zip_url: Zip URL containing data.
      num_classes: Number of labels.
      splits: Dictionary specifying the training, validation, test, etc. (key) division of data 
              (value is number of files per split).
      download_dir: Directory to download data to.

    Return:
      dir: Posix path of the resulting directories containing the splits of data.
  """
  files = list_files_per_class(zip_url)
  for f in files:
    tokens = f.split('/')
    if len(tokens) <= 2:
      files.remove(f) # Remove that item from the list if it does not have a filename

  files_for_class = get_files_per_class(files)

  classes = list(files_for_class.keys())[:num_classes]

  for cls in classes:
    new_files_for_class = files_for_class[cls]
    random.shuffle(new_files_for_class)
    files_for_class[cls] = new_files_for_class

  # Only use the number of classes you want in the dictionary
  files_for_class = {x: files_for_class[x] for x in list(files_for_class)[:num_classes]}

  dirs = {}
  for split_name, split_count in splits.items():
    print(split_name, ":")
    split_dir = download_dir / split_name
    split_files, files_for_class = split_class_lists(files_for_class, split_count)
    download_from_zip(zip_url, split_dir, split_files)
    dirs[split_name] = split_dir

  return dirs

def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
URL = 'https://storage.googleapis.com/thumos14_files/UCF101_videos.zip'
download_dir = pathlib.Path('./UCF101_subset/')
subset_paths = download_ufc_101_subset(URL, 
                        num_classes = 10, 
                        splits = {"train": 30, "test": 20}, 
                        download_dir = download_dir)

훈련 및 테스트 데이터세트를 만듭니다.

In [ ]:
batch_size = 8
num_frames = 8

output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], num_frames, training = True),
                                          output_signature = output_signature)
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], num_frames),
                                         output_signature = output_signature)
test_ds = test_ds.batch(batch_size)

여기에서 생성된 레이블은 클래스의 인코딩을 나타냅니다. 예를 들어, 'ApplyEyeMakeup'은 정수에 매핑됩니다. 훈련 데이터의 레이블을 살펴보고 데이터세트가 충분히 섞였는지 확인하세요. 

In [ ]:
for frames, labels in train_ds.take(10):
  print(labels)

데이터의 모양을 살펴봅니다.

In [ ]:
print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

## MoViNet이란?

앞서 언급한 바와 같이 [MoViNets](https://arxiv.org/abs/2103.11511)는 동작 인식과 같은 작업에서 스트리밍 비디오 또는 온라인 추론에 사용되는 비디오 분류 모델입니다. MoViNets를 사용하여 동작 인식에 비디오 데이터를 분류하는 방법을 이용해 보세요.

2D 프레임 기반 분류자는 전체 비디오에 걸쳐 실행하거나 한 번에 한 프레임씩 스트리밍하기에 효율적이고 간단합니다. 시간적 컨텍스트를 고려할 수 없기 때문에 정확도가 제한되고 프레임 간에 일관성 없는 출력을 제공할 수 있습니다.

간단한 3D CNN은 정확도와 시간적 일관성을 높일 수 있는 양방향 시간 컨텍스트를 사용합니다. 이러한 네트워크는 더 많은 리소스가 필요할 수 있으며 미래를 내다보기 때문에 스트리밍 데이터에 사용할 수 없습니다.

![표준 컨볼루션](https://www.tensorflow.org/images/tutorials/video/standard_convolution.png)

MoViNet 아키텍처는 시간 축을 따라 "인과적"인 3D 컨볼루션을 사용합니다(예: `padding="causal"`을 포함한 `layers.Conv1D`). 이것은 두 접근 방식의 장점 중 일부를 제공하며 주로 효율적인 스트리밍의 이점을 줍니다.

![인과 컨볼루션](https://www.tensorflow.org/images/tutorials/video/causal_convolution.png)

인과 컨볼루션은 시간 *t*의 출력이 시간 *t*까지의 입력만 사용하여 계산되도록 합니다. 이것이 어떻게 스트리밍을 보다 효율적으로 만들 수 있는지 보여주기 위해 친숙하고 더 간단한 예인 RNN으로 시작하겠습니다. RNN은 시간 흐름에 따라 상태를 전달합니다.

![RNN 모델](https://www.tensorflow.org/images/tutorials/video/rnn_comparison.png)

In [ ]:
gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

RNN의 `return_sequences=True` 인수를 설정하여 계산 종료 시 상태를 반환하도록 요청합니다. 그러면 일시 중지한 다음 중단한 위치에서 계속 진행하여 정확히 동일한 결과를 얻을 수 있습니다.

![RNN에서 상태 전달](https://www.tensorflow.org/images/tutorials/video/rnn_state_passing.png)

In [ ]:
first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
second_half, _ = gru(inputs[:,5:, :], initial_state=state)  # Use the state to continue where you left off.

print(np.allclose(result[:, :5,:], first_half))
print(np.allclose(result[:, 5:,:], second_half))

인과 컨볼루션은 주의해서 다루면 같은 방식으로 사용할 수 있습니다. 이 기법은 Le Paine 등이 [Fast Wavenet Generation Algorithm](https://arxiv.org/abs/1611.09482)에 사용했습니다. [MoVinet 논문](https://arxiv.org/abs/2103.11511)에서는 `state`를 "스트림 버퍼"라고 합니다.

![인과 컨볼루션에서 상태 전달](https://www.tensorflow.org/images/tutorials/video/causal_conv_states.png)

이 약간의 상태를 앞으로 전달하면 위에 표시된 전체 수용 필드를 다시 계산하는 것을 피할 수 있습니다. 

## 선행 훈련된 MoViNet 모델 다운로드

이 섹션에서는 다음을 수행합니다.

1. TensorFlow 모델에서 [`official/projects/movinet`](https://github.com/tensorflow/models/tree/master/official/projects/movinet)에 제공된 오픈 소스 코드를 사용하여 MoViNet 모델을 만들 수 있습니다.
2. 사전 훈련된 가중치를 로드합니다.
3. 미세 조정 속도를 높이기 위해 컨볼루션 베이스 또는 최종 분류자 헤드를 제외한 다른 모든 레이어를 고정합니다.

모델을 구축하려면 `a0` 구성으로 시작할 수 있습니다. 다른 모델과 비교했을 때 이것이 가장 빠른 훈련 방법이기 때문입니다. 해당 사용 사례에 적합한 모델을 확인하려면 [TensorFlow Model Garden에서 사용 가능한 MoViNet 모델](https://github.com/tensorflow/models/blob/master/official/projects/movinet/configs/movinet.py)을 확인하세요.

In [ ]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

분류자를 구축하려면 백본과 데이터세트의 클래스 수를 사용하는 함수를 만듭니다. `build_classifier` 함수는 분류자를 구축하기 위해 백본과 데이터세트의 클래스 수를 가져옵니다. 이 경우 새 분류자는 `num_classes` 출력(UCF101의 이 하위 집합에 대한 10개 클래스)을 사용합니다.

In [ ]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [ ]:
model = build_classifier(batch_size, num_frames, resolution, backbone, 10)

이 튜토리얼에서는 `tf.keras.optimizers.Adam` 옵티마이저와 `tf.keras.losses.SparseCategoricalCrossentropy` 손실 함수를 선택합니다. 모든 단계에서 모델 성능의 정확도를 보려면 metrics 인수를 사용합니다.

In [ ]:
num_epochs = 2

loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=['accuracy'])

모델을 훈련시킵니다. 두 epoch 후에 훈련 세트와 테스트 세트 모두에 대해 정확도가 높고 손실이 낮은 것에 주목하세요. 

In [ ]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=num_epochs,
                    validation_freq=1,
                    verbose=1)

## 모델 평가하기

이 모델은 훈련 데이터 세트에서 높은 정확도를 달성했습니다. 다음으로, `Model.evaluate`를 사용하여 테스트 세트에서 평가합니다.

In [ ]:
model.evaluate(test_ds, return_dict=True)

모델 성능을 더 시각화하려면 [혼동 행렬](https://www.tensorflow.org/api_docs/python/tf/math/confusion_matrix)을 사용합니다. 혼동 행렬을 사용하면 정확도를 넘어 분류 모델의 성능을 평가할 수 있습니다. 이 다중 클래스 분류 문제에 대한 혼동 행렬을 작성하기 위해 테스트세트의 실제 값과 예측 값을 가져옵니다.

In [ ]:
def get_actual_predicted_labels(dataset):
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [ ]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [ ]:
fg = FrameGenerator(subset_paths['train'], num_frames, training = True)
label_names = list(fg.class_ids_for_name.keys())

In [ ]:
actual, predicted = get_actual_predicted_labels(test_ds)
plot_confusion_matrix(actual, predicted, label_names, 'test')

## 다음 단계

이제 MoViNet 모델과 다양한 TensorFlow API(전이 학습용 API 등)를 활용하는 방법에 익숙해졌으므로 이 튜토리얼의 코드를 자신의 데이터세트에 사용해 보세요. 데이터를 비디오 데이터에 국한시킬 필요는 없습니다. MRI 스캔과 같은 체적 데이터도 3D CNN에 사용할 수 있습니다. [조현병 분류 및 조절을 위한 뇌 MRI 기반 3차원 컨볼루션 신경망](https://arxiv.org/pdf/2003.08818.pdf)에 언급된 NUSDAT 및 IMH 데이터세트는 이러한 MRI 데이터의 두 가지 소스가 될 수 있습니다.

특히, 이 튜토리얼에서 사용한 `FrameGenerator` 클래스와 다른 비디오 데이터 및 분류 튜토리얼을 사용하면 모델에 데이터를 로드하는 데 도움이 됩니다.

TensorFlow에서 비디오 데이터 작업에 대해 자세히 알아보려면 다음 튜토리얼을 확인하세요.

- [비디오 데이터 로드](https://www.tensorflow.org/tutorials/load_data/video)
- [비디오 분류를 위한 3D CNN 모델 구축](https://www.tensorflow.org/tutorials/video/video_classification)
- [스트리밍 동작 인식을 위한 MoViNet](https://www.tensorflow.org/hub/tutorials/movinet)